In [ ]:
%matplotlib inline

In [ ]:
print 'left to right:', prev, new, self.ptstart, self.ptend

In [ ]:
left to right: [299.88591767598223, 243.12342635383609] 
    [300.37024553911095, 243.01761510775461] (300.0, 250.0) (300.0, 350.0)
    
    
left to right: [299.71026968367244, 473.45196003566986] 
    [300.27775972859081, 473.82459706590669] (300.0, 250.0) (300.0, 350.0)
    
left to right: [299.25572713637376, 382.51697273695476] 
    [300.05594871441008, 383.30351288095881] (300.0, 50.0) (300.0, 150.0)
    
    
left to right: [299.80262163737905, 412.72746486919095] 
    [300.35706418484477, 413.02936260100836] (300.0, 250.0) (300.0, 350.0)


In [23]:
from copy import deepcopy
from collections import Counter
import json

from IPython.display import clear_output
from matplotlib import pyplot as plt
from scipy.stats import norm
import numpy as np

names = [
    'pidr',
    'loh',
    'uebok',
    'gnida',
    'govno',
    'zalupa',
    'penis',
    'her',
    'davalka',
    'ochko',
]

room_names = {
    '-1': 'prachechnaya',
    '1': 'hata',
    '2': 'parasha',
}

class Door(object):
    # vertical
    
    def __init__(self, ptstart, ptend, left_rn, right_rn):
        self.ptstart = ptstart
        self.ptend = ptend
        self.left_rn = left_rn
        self.right_rn = right_rn
        
        
    def check_trace(self, prev, new):
        if prev[0] == new[0]:
            return None
        dx=(prev[0]-new[0])
        a=(prev[1]-new[1])/dx

        b = prev[1] - a*prev[0]

        y_inter=self.ptstart[0]*a+b

        if self.ptstart[1] < y_inter < self.ptend[1]:
            left_to_right = (prev[0] < self.ptend[0] and new[0] > self.ptend[0])
            right_to_left = (new[0] < self.ptend[0] and prev[0] > self.ptend[0])
            if left_to_right:
                return self.right_rn
            if right_to_left:
                return self.left_rn
        return None

    
class Room(object):
    # если одна внутри другой, то сначала должна быть внешняя
    def __init__(self, area_w, area_h, left_upper, right_bottom, my_num):
        self.area_w=area_w
        self.area_h=area_h
        self.left_upper=left_upper
        self.right_bottom=right_bottom
        
        self.my_num=my_num

        self.walls = {}
        self.walls['left'] = lambda x,y: x > left_upper[0]
        self.walls['right'] = lambda x,y: x < right_bottom[0]
        self.walls['upper'] = lambda x,y: y < left_upper[1]
        self.walls['bottom'] = lambda x,y: y > right_bottom[1]

    def is_inside_room(self, x, y):
        inside = True
        for wall,checker in self.walls.items():
            inside = inside and checker(x,y)
        return inside

    
class Area(object):
    def __init__(self, w, h, people_number, speed=1.):
        self.w=w
        self.h=h
        self.people_number=people_number
        self.speed=speed
        self.X_crowd = np.random.randint(0, w, size=people_number)*1.
        self.Y_crowd = np.random.randint(0, h, size=people_number)*1.
        self.X_crowd_directions = np.zeros(people_number)
        self.Y_crowd_directions = np.zeros(people_number)
        self.rooms = []
        
    
    def init_rooms(self, rooms):
        self.rooms = rooms
        self.crowd_rooms = [
            self.get_room_number(x,y) 
            for x,y in zip(self.X_crowd, self.Y_crowd)
        ]
    
    def init_doors(self, doors):
        self.doors = doors
        
    def get_room_number(self,x,y):
        for room in self.rooms:
            ans = room.is_inside_room(x,y)
            if ans:
                return room.my_num
        return -1



    def make_iteration(self):
        prev_X = self.X_crowd.copy()
        prev_Y = self.Y_crowd.copy()
        
        addit = norm.rvs(0., self.speed, size=self.people_number)
        self.X_crowd_directions = 0.9*self.X_crowd_directions + 0.1*addit
        self.X_crowd += self.X_crowd_directions
        
        addit = norm.rvs(0., self.speed, size=self.people_number)
        self.Y_crowd_directions = 0.9*self.Y_crowd_directions + 0.1*addit
        self.Y_crowd += self.Y_crowd_directions

        self.X_crowd = [max([self.w*0.01,min([self.w*0.99,x])]) for x in self.X_crowd]
        self.Y_crowd = [max([self.h*0.01,min([self.h*0.99,y])]) for y in self.Y_crowd]
        
        
        
        new_X, new_Y = [],[]
        for p, coords in enumerate(zip(self.X_crowd, self.Y_crowd)):
            x,y = coords
            rn = self.get_room_number(x,y)
            prev_room_number = self.crowd_rooms[p]
            if not rn == prev_room_number:
                any_res = False
                for door in self.doors:
                    res = door.check_trace([prev_X[p], prev_Y[p]], [x,y])
                    if res is None:
                        continue
                    else:
                        any_res = True
                        self.crowd_rooms[p] = res
                        
                if not any_res:
                    x = prev_X[p]
                    y = prev_Y[p]
                if not (0 < x < self.w) or not (0 < y < self.h):
                    x = prev_X[p]
                    y = prev_Y[p]
            new_X += [x]
            new_Y += [y]
        self.X_crowd = np.array(new_X, dtype='float64')
        self.Y_crowd = np.array(new_Y, dtype='float64')
        
w,h=100,100

area = Area(
    w=w,
    h=h,
    people_number=10,
    speed=3.
)


rooms = [
    Room(
        area_w=w, 
        area_h=h,
        left_upper=(0,h),
        right_bottom=(w*.3,0),
        my_num=1,
    ),
    Room(
        area_w=w, 
        area_h=h,
        left_upper=(w*.6,h),
        right_bottom=(w,0),
        my_num=2,
    )
]

doors = [
    Door(
        ptstart=(30,20),
        ptend=(30,40),
        left_rn=1,
        right_rn=-1
    ),
        Door(
        ptstart=(60,60),
        ptend=(60,80),
        left_rn=-1,
        right_rn=2
    )
]



area.init_rooms(rooms)
area.init_doors(doors)




In [24]:

for i in range(1000):
    plt.figure(figsize=(9,9))
    plt.xlim((0,w))
    plt.ylim((0,h))  
    plt.xticks([])
    plt.yticks([])
    clear_output(True)    
    
    for room in area.rooms:
        lu = room.left_upper
        x1,y1 = lu
        rb = room.right_bottom
        x2,y2=rb
        #print x1,x2,y1,y2
        plt.plot([x2,x2,x1,x1,x2], [y1,y2,y2,y1,y1], c='b')

    for door in area.doors:
        plt.plot(
            [door.ptstart[0], door.ptend[0]], 
            [door.ptstart[1], door.ptend[1]], 
            c='r', 
            linewidth=3
        )

    plt.scatter(area.X_crowd, area.Y_crowd, s=50)
        
#         print 'rooms counts:', json.dumps(
#             {room_names[str(k)]: v for k,v in Counter(self.crowd_rooms).items()}, 
#             indent=4,
#         )
#         print 'people:', json.dumps(
#             dict(zip(names,[room_names[str(k)] for k in self.crowd_rooms])), 
#             indent=4,
#         )
    
    plt.show()
    
    area.make_iteration()  


KeyboardInterrupt: 